# Probabilistic Data Structures
---



## My Research

### What are Probabilistic Data Structures?

**Probabilistic data structures** are specialized structures that use **probabilistic algorithms** to estimate certain properties of the stored data. Unlike traditional data structures, which aim for precise and deterministic results, **probabilistic counterparts trade accuracy for efficiency and scalability**. They are adept at handling large datasets, where **traditional methods might be resource-intensive**.

### Advantages Over Traditional (Deterministic) Data Structures

Probabilistic data structures have notable **advantages**, including:
- **Space Efficiency:**
 - They require less memory, using probabilistic algorithms and techniques like hash functions and **Bloom filters** for compact data representation.
- **Speed and Performance:**
 - By sacrificing some accuracy, they deliver outstanding speed. They employ probabilistic algorithms that allow for quick processing of extensive datasets.
- **Memory Optimization:**
 - In-memory stores like Redis emphasize memory efficiency. Probabilistic data structures, by compressing data, help optimize memory usage, ensuring that Redis can process large datasets without sacrificing performance.

### 5 of the best Probabilistic Data Structures

These are some of the coolest ones (*for me at least*), that I've came across

#### **Bloom Filter**

##### What is **Bloom Filter**

A Bloom filter is a **space-efficient probabilistic data structure**, conceived by Burton Howard Bloom in 1970, that is used to test whether an element is a member of a set. **False positive matches are possible**, but **false negatives are not** – in other words, a query returns either "possibly in set" or "definitely not in set". Elements can be added to the set, but not removed (though this can be addressed with the counting Bloom filter variant). **The more items added, the larger the probability of false positives.**

##### How does Bloom Filter work

An empty Bloom filter is a bit array of $m$ bits, all set to **0**. It is equipped with $k$ different hash functions, which map set elements to one of the $m$ possible array positions.

- Insertion:
 - To add an element, feed it to each of the $k$ hash functions to get $k$ array positions. Set the bits at all these positions to **1**.

- Search
 - To know whether an element is in the set, feed it to each of the $k$ hash functions to get $k$ array positions. If any of the bits at these positions is **0**, the element is **definitely NOT in the set**. If all the bits are **1**, then either the element is in the set, or the bits have by chance been set to **1** during the insertion of other elements, resulting in a **false positive**.

- Deletion
 - Removing an element from Bloom filter is impossible because there is no way to tell which of the $k$ bits it maps to should be cleared. Although setting any one of those $k$ bits to zero suffices to remove the element, it would also remove any other elements that happen to map onto that bit. This would cause false negatives.


##### Advantages
While risking false positives, **Bloom filters** have a substantial space advantage over other data structures for representing sets, such as **self-balancing binary search trees, tries, hash tables** . Most of these require storing at least the data items themselves. <br>
**Bloom filters** on the other hand do not store the data items at all. The presence of an element is indicated purely by the state of the bits in the array.

 A Bloom filter with a **1%** error and an optimal value of $k$, in contrast, requires only about **9.6 bits per element**, regardless of the size of the elements.

The time needed either to add items or to check whether an item is in the set is a fixed constant, **O(k)**, completely independent of the number of items already in the set. **No other constant-space set data structure has this property.**

##### The math behind Bloom Filter

Notation:
- $m$: The number of bits in the Bloom Filter.
- $k$: The number of hash functions.
- $n$: The number of elements inserted into the Bloom Filter.

###### Probability a Certain Bit is Not Set to **1** by a Single Hash Function

- A hash function selects each array position with equal probability.
- Therefore, the probability that a specific bit is not set to **1** by one hash function during the insertion of an element is:

$$1- \frac{1}{m}$$

###### Probability a Certain Bit is Not Set to **1** by Any of the $k$ Hash Functions

- If there are **k** hash functions, the probability that a particular bit is not set to 1 by any of the **k** hash functions is:

$$\left(1 - \frac{1}{m}\right)^k$$

###### Using the Limit for $e^{-1}$

The identity for $e^{-1}$  is

$$\lim_{m\to\infty} \left(1 - \frac{1}{m}\right)^k = \frac{1}{e}$$

Applying this to our probability and for large **$m$**, we get:

$$\left(1-{\frac {1}{m}}\right)^{k}=\left(\left(1-{\frac {1}{m}}\right)^{m}\right)^{k/m}\approx e^{-k/m}$$

###### Probability a Certain Bit is Still **0** After n Insertions

After inserting $n$ elements, each with $k$ hash functions, the probability that a specific bit is still **0** is:

$$\left(1-{\frac {1}{m}}\right)^{kn}\approx e^{-kn/m}$$

Hence, the probability that a specific bit is set to **1** is:

$$ 1-\left(1-{\frac {1}{m}}\right)^{kn}\approx 1-e^{-kn/m} $$

###### False Positive Probability

- When checking membership for an element not in the set, the $k$ hash functions may all point to bits that are **1**.
- The probability of false positive is calculated with this formula:

$$ \varepsilon =\left(1-\left[1-{\frac {1}{m}}\right]^{kn}\right)^{k}\approx \left(1-e^{-kn/m}\right)^{k }$$

- This approximation assumes independence between the bits, which is not strictly true but gives a close approximation.

###### Alternative Analysis Using Fraction of Bits Set to 0

- $q$: Fraction of the $m$ bits still set to **0** after inserting $n$ items.
When testing for membership, the probability that a bit is found set to **1** by any hash function is:
$$1−q$$

- Therefore, the probability that all $k$ hash functions find their bit set to **1** is:

$$(1-q)^k$$

###### Expected Value of $q$

- The expected value of $q$ (the fraction of bits still set to **0**) is:

$$ E[q]=\left(1-{\frac {1}{m}}\right)^{kn} $$

###### Concentration of $q$

- Using the **Azuma-Hoeffding inequality**, it's shown that $q$ is strongly concentrated around its expected value:

$$ \Pr(\left|q-E[q]\right|\geq {\frac {\lambda }{m}})\leq 2\exp(-2\lambda ^{2}/kn) $$

###### False Positive Probability

- Summing over all possible values of $q$, the false positive probability is approximated as:

$$ \sum _{t}\Pr(q=t)(1-t)^{k}\approx (1-E[q])^{k}=\left(1-\left[1-{\frac {1}{m}}\right]^{kn}\right)^{k}\approx \left(1-e^{-kn/m}\right)^{k}$$

###### Optimal Number of Hash Functions

- The number of hash functions $k$ that minimizes the false positive probability for given $m$ and $n$ is:

$$ k={\frac {m}{n}}\ln 2 $$

###### Required Number of Bits

- To achieve a desired false positive probability $\varepsilon$, the required number of bits $m$ can be computed as follows:



1.   Substituting the optimal $k$ into the false positive probability expression:

$$ \varepsilon =\left(1-e^{-({\frac {m}{n}}\ln 2){\frac {n}{m}}}\right)^{{\frac {m}{n}}\ln 2}=\left({\frac {1}{2}}\right)^{{\frac {m}{n}}\ln 2}$$

2. Simplifying the above expression:

$$ \ln(\varepsilon )=-{\frac {m}{n}}\ln(2)^{2}$$

$$ {\frac {m}{n}}=-{\frac {\ln(\varepsilon )}{\ln(2)^{2}}}\approx -2.08\ln(\varepsilon ) $$

3. The corresponding number of hash functions (ignoring integrality):

$$ k=-{\frac {\ln(\varepsilon )}{\ln(2)}}$$

###### Union and Intersection of Sets

- Bloom Filters can be used to estimate the sizes of the union and intersection of two sets.

- Estimate Count of Bloom Filters:
 - For Bloom Filter $A$:

 $$ n(A^{*})=-{\frac {m}{k}}\ln \left[1-{\frac {|A|}{m}}\right] $$

 - For Bloom Filter $B$

 $$ n(B^{*})=-{\frac {m}{k}}\ln \left[1-{\frac {|B|}{m}}\right] $$

- Size of Their Union:

$$ n(A^{*}\cup B^{*})=-{\frac {m}{k}}\ln \left[1-{\frac {|A\cup B|}{m}}\right] $$
*where $ n(A \cup B) $ is the number of bits set to one in either Bloom Filter.*

- Size of Their Intersection:

$$ n(A^{*}\cap B^{*})=n(A^{*})+n(B^{*})-n(A^{*}\cup B^{*} ) $$

##### Code example

In [ ]:
!pip install bitarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 1.9 MB/s eta 0:00:00


In [ ]:
import hashlib
import math
from bitarray import bitarray

class BloomFilter:
    def __init__(self, num_items, false_positive_rate):
        self.num_items = num_items
        self.false_positive_rate = false_positive_rate

        # Calculating the size of the bit array (m) and number of hash functions (k)
        self.size = -int((num_items * math.log(false_positive_rate)) / (math.log(2) ** 2))
        self.num_hashes = int((self.size / num_items) * math.log(2))

        # Initializing bit array to all 0's
        self.bit_array = bitarray(self.size)
        self.bit_array.setall(0)

    def _hashes(self, item):
        """Generate k hash values for the given item."""
        hash1 = int(hashlib.md5(item.encode()).hexdigest(), 16)
        hash2 = int(hashlib.sha1(item.encode()).hexdigest(), 16)
        for i in range(self.num_hashes):
            yield (hash1 + i * hash2) % self.size

    def add(self, item):
        """Add an item to the Bloom Filter."""
        for index in self._hashes(item):
            self.bit_array[index] = 1

    def contains(self, item):
        """Check if an item is in the Bloom Filter."""
        return all(self.bit_array[index] for index in self._hashes(item))

    def estimate_items(self):
        """Estimate the number of items in the Bloom Filter."""
        X = self.bit_array.count(1)
        m = self.size
        k = self.num_hashes
        return - (m / k) * math.log(1 - X / m)

    def union(self, other):
        """Return a new Bloom Filter that represents the union of this and another Bloom Filter."""
        if self.size != other.size or self.num_hashes != other.num_hashes:
            raise ValueError("Bloom Filters must have the same size and number of hash functions to union")
        result = BloomFilter(self.num_items, self.false_positive_rate)
        result.bit_array = self.bit_array | other.bit_array
        return result

    def intersection(self, other):
        """Return a new Bloom Filter that represents the intersection of this and another Bloom Filter."""
        if self.size != other.size or self.num_hashes != other.num_hashes:
            raise ValueError("Bloom Filters must have the same size and number of hash functions to intersect")
        result = BloomFilter(self.num_items, self.false_positive_rate)
        result.bit_array = self.bit_array & other.bit_array
        return result

bf = BloomFilter(num_items=100, false_positive_rate=0.01)

items_to_add = ["apple", "banana", "cherry", "cucumber", "tomato"]
for item in items_to_add:
    bf.add(item.lower())

# Check
items_to_check = ["Apple", "banana", "peach", "grape"]
for item in items_to_check:
    print(f"{item} in Bloom Filter: {bf.contains(item.lower())}")

# Estimating the number of items in the Bloom Filter
print(f"Estimated number of items: {bf.estimate_items()}")

# Creating another Bloom Filter and perform union and intersection
bf2 = BloomFilter(num_items=100, false_positive_rate=0.01)
other_items_to_add = ["peach", "grape", "honeydew", "banana"]
for item in other_items_to_add:
    bf2.add(item.lower())

# Union
union_bf = bf.union(bf2)
print(f"Estimated number of items in union: {union_bf.estimate_items()}")

# Intersection
intersection_bf = bf.intersection(bf2)
print(f"Estimated number of items in intersection: {intersection_bf.estimate_items()}")


Apple in Bloom Filter: True
banana in Bloom Filter: True
peach in Bloom Filter: False
grape in Bloom Filter: False
Estimated number of items: 5.0799618811507035
Estimated number of items in union: 8.032012620174063
Estimated number of items in intersection: 1.1709498962667566


#### **Skip List**

##### What is **Skip List**

A skip list is a probabilistic data structure that provides efficient $O(\log{n})$ average time complexity for search and insertion operations within an ordered sequence of $n$ elements. It combines the best features of a **sorted array** and a **linked list**, allowing for fast search operations and dynamic insertion capabilities.

##### Structure

- **Layers**: A skip list is composed of multiple layers of **linked lists**. The bottom layer (Layer 1) is a standard ordered linked list containing all the elements. Each higher layer acts as an "express lane", skipping over more elements.
- **Probability**: An element in layer $i$ appears in layer $i+1$ with a fixed probability $p$. Common values for $p$ are $1/2$ or $1/4$. This means, on average, each element appears in $1/(1-p)$ lists.
- **Height**: The expected number of layers is $\log_{1/p}{n}$, where $n$ is the number of elements.

##### Search

- Start at the **topmost layer** and move **horizontally** until the current element is **greater than or equal to the target**. If it is equal, the search is successful. **If greater, drop down to the next lower layer and continue the search**.

##### Insertion

- Similar to search, but elements are inserted at appropriate positions in multiple layers based on the probability $p$.

##### The math behind Skip List

There is not much, since it is a pretty simple data structure.

- **Search Complexity**: The expected number of steps in each layer is $1/p$, leading to a total expected search cost of $1/p * \log_{1/p} {n}$, which simplifies to $O(\log{n})$ for constant $p$.

- **Insertion Complexity**: Insertion also has an average complexity of $O(\log {n})$.

##### Code Example

In [ ]:
import random

class Node:
    def __init__(self, value, level):
        self.value = value
        self.forward = [None] * (level + 1)

class SkipList:
    def __init__(self, max_level, p):
        self.max_level = max_level
        self.p = p
        self.header = self.create_node(self.max_level, -1)
        self.level = 0

    def create_node(self, level, value):
        return Node(value, level)

    def random_level(self):
        level = 0
        while random.random() < self.p and level < self.max_level:
            level += 1
        return level

    def insert(self, value):
        update = [None] * (self.max_level + 1)
        current = self.header

        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]
            update[i] = current

        level = self.random_level()

        if level > self.level:
            for i in range(self.level + 1, level + 1):
                update[i] = self.header
            self.level = level

        new_node = self.create_node(level, value)

        for i in range(level + 1):
            new_node.forward[i] = update[i].forward[i]
            update[i].forward[i] = new_node

    def search(self, value):
        current = self.header

        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].value < value:
                current = current.forward[i]

        current = current.forward[0]

        if current and current.value == value:
            return True
        return False

    def display_list(self):
        for i in range(self.level + 1):
            print(f"Level {i}: ", end="")
            node = self.header.forward[i]
            while node:
                print(node.value, end=" ")
                node = node.forward[i]
            print("")

max_level = 4
p = 0.5
skiplist = SkipList(max_level, p)

elements = [3, 6, 7, 9, 12, 19, 17, 26, 21]
for elem in elements:
    skiplist.insert(elem)

skiplist.display_list()

search_elements = [19, 6, 21, 25]
for elem in search_elements:
    found = skiplist.search(elem)
    print(f"Element {elem} {'found' if found else 'not found'} in the skip list")


Level 0: 3 6 7 9 12 17 19 21 26 
Level 1: 3 7 12 17 
Level 2: 17 
Level 3: 17 
Element 19 found in the skip list
Element 6 found in the skip list
Element 21 found in the skip list
Element 25 not found in the skip list


#### **HyperLogLog**

##### What is **HyperLogLog**

HyperLogLog is a probabilistic algorithm designed to estimate the **number of distinct elements** in a multiset. It is particularly effective for large datasets where storing exact counts would be impractical. This algorithm uses significantly less memory than traditional methods while maintaining a high level of accuracy.

##### Advantages of using HyperLogLog

- **Memory Efficiency**: HyperLogLog can estimate cardinalities over **109109** with typical accuracy (standard error) of around **2%**, using just **1.5 KB of memory**.

- **Origin**: It builds on the earlier **LogLog** algorithm and the Flajolet-Martin algorithm from 1984.

- **Accuracy**: Achieves high accuracy with a relative error of about $1.04/m$
​, where $m$ is the number of registers.

##### How does it work?

The core idea is to estimate the cardinality of a multiset by **evaluating the maximum number of leading zeros in the binary representation of each element**. For a multiset of uniformly distributed random numbers:

- **Estimate**: If the maximum number of leading zeros in the binary representation of the elements is $n$, the estimate for the number of distinct elements is $2^n$.

- **Hashing**: Apply a hash function to each element to obtain a uniformly distributed random number.
- **Binary Analysis**: For each hashed value, determine the number of leading zeros. The maximum count of leading zeros across all elements is used to estimate the cardinality.

##### Operations and mathematical analysis

Legend:
- $v$ => input data
- $h$ => hash function
- $b$ => bits, where $b$ is $\log _{2}(m)$
- $ \rho (w)$ => number of leading zeros
- $M$ => Array of $m$ counters (or "registers") that are initialized to **0**

###### Addition:

- **Hashing**:
 - Compute the hash $x=h(v)$

- **Register Index**:
 - Determine the register $j$ using the first $b$ bits $j=1+\langle x_{1}x_{2}...x_{b}\rangle _{2}$

- **Leading Zeros**:
 - Compute $ρ(w)$, where $w$ is the remaining bits after the first $b$.

- **Update**:
 - Set $M[j]=max(M[j],\rho (w))$

Mathematically speaking:
$$\begin{aligned}x&:=h(v)\\j&:=1+\langle x_{1}x_{2}...x_{b}\rangle _{2}\\w&:=x_{b+1}x_{b+2}...\\M[j]&:=\max(M[j],\rho (w))\\\end{aligned}$$

###### Count operation:

- Estimate the cardinality using the harmonic mean of the registers:

$$ Z={\Bigg (}\sum _{j=1}^{m}{2^{-M[j]}}{\Bigg )}^{-1}$$

- The constant $\alpha_m$ corrects the bias:

$$ \alpha _{m}=\left(m\int _{0}^{\infty }\left(\log _{2}\left({\frac {2+u}{1+u}}\right)\right)^{m}\,du\right)^{-1}$$

- The cardinality estimate $E$ is:

$$E=\alpha _{m}m^{2}Z$$

###### Practical Considerations


- **Bias Correction**: The constant $\alpha_m$ is approximated as:

 $$
  \alpha _{m}\approx {\begin{cases}0.673,&{\text{for }}m=16;\\0.697,&{\text{for }}m=32;\\0.709,&{\text{for }}m=64;\\{\frac {0.7213}{1+1.079/m}},&{\text{for }}m\geq 128.\end{cases}}
 $$

- **Small Cardinalities**: For cardinalities below $\frac {5}{2}m$, use Linear Counting:

$$ E^{\star }=m\log \left({\frac {m}{V}}\right) $$ where $V$ is the count of zero registers.

- **Large Cardinalities**: For cardinalities exceeding $\frac{2^{32}}{30}$:

$$ E^{\star }=-2^{32}\log \left(1-{\frac {E}{2^{32}}}\right)$$

- **Error Estimation**: The standard deviation of the estimate is given by:

$$\sigma =\frac{1.04}{\sqrt {m}}$$

###### Merge operation:

To combine two HyperLogLogs:

$${\mathit {hll}}_{\text{union}}[j]=\max({\mathit {hll}}_{1}[j],{\mathit {hll}}_{2}[j])$$

##### Complexity Analysis

- **Memory**:
 - Uses $ O(\epsilon ^{-2}\log \log n+\log n) $  space, where $n$ is the number of elements and $m$ is the number of registers.

- **Add Operation**:
 - $O(1)$, given a fixed-size hash output.

- **Count and Merge**:
 - Typically $O(m)$, with some implementations considering it $O(1)$

##### Code Example

In [ ]:
import hashlib
import math

class HyperLogLog:
    def __init__(self, m):
        self.m = m
        self.registers = [0] * m
        self.alpha_m = self.get_alpha(m)

    def get_alpha(self, m):
        if m == 16:
            return 0.673
        elif m == 32:
            return 0.697
        elif m == 64:
            return 0.709
        else:
            return 0.7213 / (1 + 1.079 / m)

    def add(self, value):
        x = int(hashlib.md5(value.encode('utf8')).hexdigest(), 16)
        b = int(math.log2(self.m))
        j = (x >> (128 - b)) & (self.m - 1)
        w = x & ((1 << (128 - b)) - 1)
        self.registers[j] = max(self.registers[j], self.rho(w))

    def rho(self, w):
        bin_w = bin(w)[2:]
        return 1 + len(bin_w) - len(bin_w.lstrip('0'))

    def count(self):
        Z = 1.0 / sum([2 ** (-reg) for reg in self.registers])
        E = self.alpha_m * self.m * self.m * Z
        V = sum(1 for reg in self.registers if reg == 0)
        if E < (5 / 2) * self.m:
            return self.m * math.log(self.m / V)
        else:
            return E

    def merge(self, other):
        for j in range(self.m):
            self.registers[j] = max(self.registers[j], other.registers[j])

hll1 = HyperLogLog(4096)
hll2 = HyperLogLog(4096)

for elem in ['apple', 'banana', 'cherry', 'date', 'peach', 'grape', 'melon', 'fig', 'kiwi', 'pineapple', 'apricot','apple']: # 12 elements with repeating apple
  hll1.add(elem)
  hll2.add(elem)

hll1.merge(hll2)

print(f"Estimated cardinality: {hll1.count()}")

Estimated cardinality: 11.014797005785086


#### **Count–min sketch**

Count-Min Sketch is a probabilistic data structure used for **efficiently counting the frequency of events in a data stream**. It provides approximate counts with a guaranteed error bound using **sub-linear space**. Invented by Graham Cormode and S. Muthu Muthukrishnan in 2003, it is particularly useful for large-scale data processing where exact counting is impractical.

##### Advantages of using Count-min sketch

- **Probabilistic**: Offers approximate counts with a controlled error margin.
- **Space-Efficient**: Requires sub-linear space relative to the number of unique elements.
- **Mergeable**: Sketches from different data streams can be combined.

##### Structure

The **Count-Min Sketch** consists of a two-dimensional array (table) with $d$ rows and $w$ columns. Each row is associated with a **unique hash function**. The parameters $w$ and $d$ are chosen based on the desired error bounds:

- $w=\lceil e/\epsilon \rceil$: Determines the number of columns, related to the approximation error $\varepsilon$.
- $d=\lceil \ln⁡(1/\delta) \rceil$: Determines the number of rows, related to the confidence level $1-\delta$.

##### Adding an Event

When a new event of type $i$ arrives:

- For each row $j$:

 - Compute the column index $k=h_j(i)$ using the hash function for that row.
 - Increment the value in row $j$, column $k$ by **1**.

##### Universe of event types
The universe of event types $\mathcal{U} $ refers to the **set of all possible distinct events that could appear in the data stream**. At any given time, the sketch can be queried for the frequency of a particular event type $i$ from this universe. The sketch returns an estimate of the frequency that is within a certain distance of the true frequency, with a specified probability.

*For example, if you are monitoring network traffic, the universe of event types could be all possible IP addresses that might appear in the traffic. If you are counting words in a large collection of documents, the universe could be all possible words.*

###### Querying Frequency

To estimate the frequency of an event type $i$:

1. Compute the column index for each row using the hash functions.
2. Return the minimum value from these columns.

The estimate $\hat{a}_i​$ of the true frequency $\hat{a}_i​$​ satisfies:

$$a_i \le \hat{a}_i \le a_i + \epsilon N$$

with probability $1−\delta$, where $N$ is the total number of events processed.

$$N=\sum_{i\in {\mathcal {U}}}a_{i}$$

##### Mathematical Analysis

###### Error Bounds

The Count-Min Sketch guarantees that the estimated frequency $\hat{a} _i$ of an event $i$ is within $\epsilon N$ of the true frequency $a_i$​ with probability $1 - \delta$:

$$\hat{a}_i \le a_i + \epsilon N$$

###### Parameter Selection

- **Width** $w$: Controls the approximation error $\epsilon$.

$$w=\lceil e/\epsilon \rceil$$

- **Depth** $\delta$: Controls the confidence level $1 - \delta$.

$$d=\lceil \ln⁡(1/\delta) \rceil$$

##### Complexity

- Time complexity:
 - **Insert(key)**: $O(d)$
 - **Query(key)**: $O(d)$

Both are constant, considering the number of hashing functions would be constant.

- Space complexity
 - $O(wd)$, it can be considered constant as well because it does not essentially depend on the number of keys inserted.

 Given the parameters $w$ and $d$, the space required is significantly less than a traditional hash table, making it suitable for large-scale data processing.

##### Practical consideration

- **Bias and Error Reduction**: Several techniques, such as hCount* and Maximum Likelihood Estimation (MLE), can be used to reduce bias and improve accuracy.

- **Mergeability**: Count-Min Sketches can be merged by element-wise addition, making it useful for distributed data processing.

##### Code Example

In [ ]:
import numpy as np
import random
import hashlib

class CountMinSketch:
    def __init__(self, width, depth):
        self.width = width
        self.depth = depth
        self.table = np.zeros((depth, width), dtype=int)
        self.hashes = [self._hash_function(seed) for seed in range(depth)]

    def _hash_function(self, seed):
        def hash_fn(x):
            return int(hashlib.md5((str(seed) + str(x)).encode()).hexdigest(), 16) % self.width
        return hash_fn

    def add(self, item):
        for i, hash_fn in enumerate(self.hashes):
            self.table[i, hash_fn(item)] += 1

    def query(self, item):
        return min(self.table[i, hash_fn(item)] for i, hash_fn in enumerate(self.hashes))

cms = CountMinSketch(width=2000, depth=5)
events = ['apple', 'banana', 'apple', 'pineapple', 'banana', 'apple']

for event in events:
    cms.add(event)

print(f"Estimated count for 'apple': {cms.query('apple')}")
print(f"Estimated count for 'banana': {cms.query('banana')}")
print(f"Estimated count for 'pineapple': {cms.query('pineapple')}")
print(f"Estimated count for 'peach': {cms.query('peach')}")

for _ in range(100_000):
    cms.add(random.choice(events))

print(f"100 000 random items added")
print(f"Estimated count for 'apple': {cms.query('apple')}")
print(f"Estimated count for 'banana': {cms.query('banana')}")
print(f"Estimated count for 'pineapple': {cms.query('pineapple')}")
print(f"Estimated count for 'peach': {cms.query('peach')}")


Estimated count for 'apple': 3
Estimated count for 'banana': 2
Estimated count for 'pineapple': 1
Estimated count for 'peach': 0
100 000 random items added
Estimated count for 'apple': 50037
Estimated count for 'banana': 33161
Estimated count for 'pineapple': 16808
Estimated count for 'peach': 0


#### **Quotient Filter**

**Quotient Filter** is a space-efficient probabilistic data structure introduced by Michael Bender and colleagues in 2011. It is used to manage a set of elements and supports four key operations:
- **Adding** elements
- **Deleting** elements
- **Checking membership**
- **Checking non-membership**

It offers benefits similar to **Bloom filters** but with different trade-offs in terms of space and performance.

##### Advantages of using **Quotient filter**

- **Space-Efficient**: Uses less space compared to traditional hash tables.
- **Probabilistic**: May produce false positives but never false negatives.
- **Supports Deletion**: Unlike Bloom filters, quotient filters can efficiently delete elements.
- **Mergeable**: Multiple quotient filters can be merged without affecting their false positive rates.

##### Structure:

The **quotient filter** relies on two parameters:
- $p$: Size (in bits) for fingerprints.
- $q$: Determines the number of buckets $(m=2^q)$ in the hash table.

The filter uses one hash function that generates $p$-bit fingerprints for the elements. These fingerprints are divided into:
- Quotient $(f_q)$: The most significant $q$ bits.
- Remainder $(f_r)$: The least significant $p−q$ bits.

Each bucket in the hash table contains three status bits:
- **is_occupied**: Indicates if the bucket is the canonical bucket (home position) for some fingerprint.
- **is_continuation**: Indicates if the bucket is part of a run but not the first bucket of the run.
- **is_shifted**: Indicates if the fingerprint in the bucket has been shifted from its canonical position.

##### Operations

###### Adding an Element

1. Hash the element to produce fingerprint $f$
2. Calculate the quotient $f_q$ and remainder $f_r$ from the fingerprint.
3. Locate the canonical bucket using the quotient.
4. Insert the remainder f_r into the correct position within the run, maintaining sorted order.
5. Shift elements if necessary to maintain the contiguous run structure.

###### Deleting an Element

1. Locate the canonical bucket using the quotient.
2. Find the run that the element belongs to.
3. Remove the element by adjusting the remainders and status bits appropriately.

###### Testing Membership

1. Hash the element to produce fingerprint $f$
2. Calculate the quotient $f_q$ and remainder $f_r$
3. Check the canonical bucket for occupancy.
4. Scan the run if the canonical bucket is occupied, comparing stored remainders with $f_r$
5. Determine membership based on the comparison:
  - If a matching remainder is found, the element is probably in the set.
  - If not found, the element is definitely not in the set.

##### Mathematical Analysis

###### False Positives and False Negatives

- **False Positives**: The quotient filter may report an element as present when it is not. The probability of a false positive is given by:

$$P(\text{false positive}) \le \frac{1}{2^r}$$

where $r$ is the number of bits in the remainder.

- **False Negatives**: Quotient filters do not produce false negatives. If the filter reports that an element is not present, it is **definitely NOT** in the set.

###### Space Complexity

The space complexity is determined by the number of buckets $m=2^q$ and the size of the fingerprints $p$

$$Space=m×(\text{bits per bucket})$$

where each bucket typically requires a few status bits plus the remainder bits.

##### Practical Considerations

- **Efficiency**: Quotient filters are generally faster than **Bloom filters** because they only require evaluating a single hash function per operation.

- **Mergeability**: Two quotient filters can be merged efficiently without affecting their false positive rates, which is not possible with Bloom filters.

- **Deletion**: The quotient filter supports element deletion, which **Bloom filters do not support natively**.


##### Comparison with Bloom Filter

- **Space Usage**: Quotient filters are about **20% larger** than **Bloom filters**.
- **Performance**: Quotient filters have **faster insert and lookup times** due to requiring fewer hash function evaluations and cache misses.
- **Deletion**: Quotient filters **support deletion** of elements, while **Bloom filters** do not.

##### Code example

In [ ]:
class QuotientFilter:
    def __init__(self, q, p):
        self.q = q
        self.p = p
        self.m = 2 ** q
        self.table = [{'occupied': False, 'continuation': False, 'shifted': False, 'remainder': None} for _ in range(self.m)]

    def _hash(self, x):
        return hash(x) % (2 ** self.p)

    def _get_quotient_remainder(self, f):
        quotient = f >> (self.p - self.q)
        remainder = f & ((1 << (self.p - self.q)) - 1)
        return quotient, remainder

    def add(self, x):
        f = self._hash(x)
        fq, fr = self._get_quotient_remainder(f)
        self._insert(fq, fr)

    def _insert(self, fq, fr):
        pos = fq
        while self.table[pos]['occupied']:
            if pos == fq:
                self.table[pos]['shifted'] = True
            pos = (pos + 1) % self.m

        self.table[pos] = {'occupied': True, 'continuation': False, 'shifted': pos != fq, 'remainder': fr}

    def contains(self, x):
        f = self._hash(x)
        fq, fr = self._get_quotient_remainder(f)
        return self._find(fq, fr)

    def _find(self, fq, fr):
        pos = fq
        while self.table[pos]['occupied']:
            if self.table[pos]['remainder'] == fr:
                return True
            if not self.table[pos]['continuation']:
                break
            pos = (pos + 1) % self.m
        return False

qf = QuotientFilter(q=3, p=5)
elements = ['amsterdam', 'berlin', 'london', 'madrid', 'ankara', 'abu dhabi']

for element in elements:
    qf.add(element)

print(f"Contains 'berlin': {qf.contains('berlin')}")
print(f"Contains 'paris': {qf.contains('paris')}")


Contains 'berlin': True
Contains 'paris': False


### More Probabilistic Data Structures

These are some more cool ones that I came across while researching.

#### **Cuckoo filter**

A **cuckoo filter** is a space-efficient probabilistic data structure that is used to test whether an element is a member of a set, like a **Bloom filter** does.

- Similar to **Bloom Filter** but supports deletion and is more space-efficient for certain scenarios.
- Uses **cuckoo hashing** to handle collisions.

#### **LogLog**

Yes, as you may have thought, this is similar to **HyperLogLog**.

- Predecessor to **HyperLogLog** for cardinality estimation.
- Less accurate but simpler.

#### **T-digest**

The t-digest is an on-line algorithm for building **small sketches of data** that can be used to **approximate rank-based statistics** with high accuracy, particularly near the tails.

It may help you find solutions to such questions:

- Which fraction of the values in the data stream are smaller than a given value?
- How many values in the data stream are smaller than a given value?
- Which value is smaller than $p$ percent of the values in the data stream? What is the p-percentile value?
- What is the mean value between the $p1$-percentile value and the $p2$-percentile value?
- What is the value of the $n^{th}$ smallest (or largest) value in the data stream? What is the value with [reverse] rank $n$?


## Sources

- [Wikipedia.org](https://en.wikipedia.org/wiki/HyperLogLog)
- [Brilliant.org](https://brilliant.org/wiki/bloom-filter)
- [Medium.com](https://medium.com/@rrinlondon/count-min-sketch-5bf22743798a)
- [Redis.io](https://redis.io/blog/t-digest-in-redis-stack/)
- [Github.com](https://github.com/PetarWho)
- [Youtube.com](https://www.youtube.com/)
- [GeeksForGeeks.org](https://www.geeksforgeeks.org/bloom-filters-introduction-and-python-implementation/)
- [cs.cmu.edu](https://www.cs.cmu.edu/~dga/papers/cuckoo-conext2014.pdf)